# MSMARCO Document Ranking Task using PyTerrier - uogTrBaseDPHQ

This notebook demonstrates indexing and performing a baseline DPH + Query Expansion run for the MSMARCO Document Ranking task using [PyTerrier](https://github.com/terrier-org/pyterrier).

Author: Craig Macdonald, University of Glasgow

## PyTerrier Setup

We need to install PyTerrier. We can do this using Pip by uncommenting this line.

In [1]:
#!pip install --upgrade git+https://github.com/terrier-org/pyterrier.git#egg=python-terrier

If your JAVA_HOME environment variable does not specify a directory for Java 11, you should set it here.

In [2]:
import os
os.environ["JAVA_HOME"] = "/local/trmaster/opt/jdk-11.0.6/"

Lets now start PyTerrier.

In [3]:
import pyterrier as pt
if not pt.started():
  pt.init(mem=8000)

## Dataset Setup

PyTerrier contains a Datasets API that alllows to index/retrieve from a number of standard datasets. We can see which datasets are supported using `pt.list_datasets()`:

In [4]:
pt.list_datasets()

,dataset,topics,qrels,corpus,index
0,50pct,None,None,None,True
1,vaswani,True,True,True,True
2,trec-deep-learning-docs,"(train, dev, test, test-2020, leaderboard-2020)","(train, dev, test)",True,None
3,trec-robust-2004,True,True,None,None
4,trec-robust-2005,True,True,None,None
5,trec-covid,"(round1, round2, round3)","(round1, round2)",None,None
6,trec-wt2g,True,True,None,None
7,trec-wt-2002,"(td, np)","(np, td)",None,None
8,trec-wt-2003,"(td, np)","(np, td)",None,None
9,trec-wt-2004,"(all, np, hp, td)","(hp, td, np, all)",None,None


For the MSMARCO document ranking task, the corresponding dataset is `"trec-deep-learning-docs"`, which we can see provides various topics and qrels sets, and provides a copy of the corpus.

In [5]:
dataset = pt.get_dataset("trec-deep-learning-docs")

If we run `get_corpus()`, it will download the TREC formatted version of the corpus. NB: This is 22GB, so too much for Google Colab unfortunately.

In [6]:
dataset.get_corpus()

['/users/craigm/.pyterrier/corpora/trec-deep-learning-docs/corpus/msmarco-docs.trec.gz']

## Indexing

Lets get setup for indexing. This is a basic configuration, without applying any stopword removal or stremming. Indexing on our machine took just over 1 hour using a single thread.

In [7]:
if not os.path.exists("./index/data.properties"):

    !mkdir -p index
    props = {
      'indexer.meta.reverse.keys':'docno',
      'termpipelines' : '',
    }

    pt.logging('INFO')
    indexer = pt.TRECCollectionIndexer("./index")
    indexer.setProperties(**props)
    indexref = indexer.index(dataset.get_corpus())
    
else:
    indexref = pt.IndexRef.of("./index/data.properties")

Lets see the statistics of the generated index.

In [8]:
pt.logging('WARN')
index = pt.IndexFactory.of(indexref)
print(index.getCollectionStatistics().toString())


Number of documents: 3213835
Number of terms: 17470544
Number of fields: 0
Number of tokens: 3667907097
Field names: []
Positions:   false



All being well, you should have indexed 3213835 documents

## Retrieval

This notebook contains a demonstration of how to execute a baseline retrieval run, using a Divergence from Randomness weighting model called [DPH](http://terrier.org/docs/current/javadoc/org/terrier/matching/models/DPH.html) and the Bo1 query expansion model.

We use an object called BatchRetrieve. The constructor parameters are as follows:
 - `wmodel` - name of the Terrier weighting model class
 - `control` - Retrieval configuration - here we specify the query expansion configuration
 - `properties` - Terrier global configuration - here we re-specify the termpipeline to match the indexing configuration
 - `verbose` - we set this to True, so we can view progress (using [TQDM](https://github.com/tqdm/tqdm)) when retrieving for these large topic sets.
 
Finally, we only want 100 results per query, so we apply the rank cutoff operator `%`.

In [14]:
DPH_br_qe = pt.BatchRetrieve(index, 
                             wmodel="DPH", 
                             controls={"qe" : "on", "qemodel" : "Bo1"}, 
                             properties={"termpipelines": ""}, 
                             verbose=True) % 100

Lets now evaluate performance on the MSMARCO Dev set. Experiment is a declarative notation for running one or more experiment pipelines on a standard set of topics, then evaluating them for the same qrels. We report the MRR measure.

The dev set is quite large (> 5000 queries). This took abuot 2-3 hours to run for us.

In [10]:
pt.Experiment([DPH_br_qe], dataset.get_topics("dev"), dataset.get_qrels("dev"), eval_metrics=["recip_rank"])

18:17:43.409 [main] WARN  o.t.a.batchquerying.TRECQuery - trec.encoding is not set; resorting to platform default (ISO-8859-1). Retrieval may be platform dependent. Recommend trec.encoding=UTF-8


100%|██████████| 5193/5193 [2:13:43<00:00,  1.55s/q]  


,name,recip_rank
0,"RankCutoff(BR(DPH), 100)",0.248632


## Prepare Leaderboard results

Finally, lets prepare a results file for sending to the leaderboard. Again, with 5793 topics, this took about 2-3 hours.

In [15]:
pt.io.write_results(DPH_br_qe(dataset.get_topics("leaderboard-2020")), "uogTrBaseDPHQ.res.gz", format="minimal")

01:38:03.439 [main] WARN  o.t.a.batchquerying.TRECQuery - trec.encoding is not set; resorting to platform default (ISO-8859-1). Retrieval may be platform dependent. Recommend trec.encoding=UTF-8


100%|██████████| 5793/5793 [2:39:48<00:00,  1.66s/q]  
